# Natural Language Processing is Fun!
## An NLP workshop by Adam Geitgey

* References:

https://medium.com/@ageitgey/natural-language-processing-is-fun-9a0bff37854e

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/learning-stack/Colab-ML-Playbook/blob/master/NLP/Natural%20Language%20Processing%20is%20Fun/NLP_Is_Fun.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/learning-stack/Colab-ML-Playbook/blob/master/NLP/Natural%20Language%20Processing%20is%20Fun/NLP_Is_Fun.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

**Building an NLP Pipeline, Step-by-Step**
Step 1: Sentence Segmentation
Step 2: Word Tokenization
Step 3: Predicting Parts of Speech for Each Token
Step 4: Text Lemmatization
Step 5: Identifying Stop Words
Step 6: Dependency Parsing
Step 6b: Finding Noun Phrases
Step 7: Named Entity Recognition (NER)
Step 8: Coreference Resolution
![alt text](https://cdn-images-1.medium.com/max/1400/1*zHLs87sp8R61ehUoXepWHA.png)

In [2]:
# Install spaCy 
!pip3 install -U spacy

# Download the large English model for spaCy
!python3 -m spacy download en_core_web_lg

# Install textacy which will also be useful
!pip3 install -U textacy

Requirement already up-to-date: spacy in /usr/local/lib/python3.6/dist-packages (2.0.18)
    100% |████████████████████████████████| 13.9MB 3.2MB/s 
featuretools 0.4.1 has requirement pandas>=0.23.0, but you'll have pandas 0.22.0 which is incompatible.
cufflinks 0.14.6 has requirement plotly>=3.0.0, but you'll have plotly 1.12.12 which is incompatible.
  Found existing installation: numpy 1.14.6
    Uninstalling numpy-1.14.6:
      Successfully uninstalled numpy-1.14.6
    100% |████████████████████████████████| 852.3MB 58.1MB/s 
  Running setup.py install for en-core-web-lg ... - \ | / - \ | / done

    Linking successful
    /usr/local/lib/python3.6/dist-packages/en_core_web_lg -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/en_core_web_lg

    You can now load the model via spacy.load('en_core_web_lg')

    100% |████████████████████████████████| 143kB 13.5MB/s 
    100% |████████████████████████████████| 3.0MB 11.8MB/s 
    100% |██████████████████████████

In [3]:
import spacy

# Load the large English NLP model
nlp = spacy.load('en_core_web_lg')

# The text we want to examine
text = """London is the capital and most populous city of England and 
the United Kingdom.  Standing on the River Thames in the south east 
of the island of Great Britain, London has been a major settlement 
for two millennia. It was founded by the Romans, who named it Londinium.
"""

# Parse the text with spaCy. This runs the entire pipeline.
doc = nlp(text)

# 'doc' now contains a parsed version of text. We can use it to do anything we want!
# For example, this will print out all the named entities that were detected:
for entity in doc.ents:
    print(f"{entity.text} ({entity.label_})")

London (GPE)
England (GPE)
the United Kingdom (GPE)
the River Thames (FAC)
Great Britain (GPE)
London (GPE)
two millennia (DATE)
Romans (NORP)
Londinium (PERSON)



TYPE	DESCRIPTION

PERSON	People, including fictional.

NORP	Nationalities or religious or political groups.

FAC	Buildings, airports, highways, bridges, etc.

ORG	Companies, agencies, institutions, etc.

GPE	Countries, cities, states.

LOC	Non-GPE locations, mountain ranges, bodies of water.

PRODUCT	Objects, vehicles, foods, etc. (Not services.)

EVENT	Named hurricanes, battles, wars, sports events, etc.

WORK_OF_ART	Titles of books, songs, etc.

LAW	Named documents made into laws.

LANGUAGE	Any named language.

DATE	Absolute or relative dates or periods.

TIME	Times smaller than a day.

PERCENT	Percentage, including "%".

MONEY	Monetary values, including unit.

QUANTITY	Measurements, as of weight or distance.

ORDINAL	"first", "second", etc.

CARDINAL	Numerals that do not fall under another type.

In [4]:
# Replace a token with "REDACTED" if it is a name
def replace_name_with_placeholder(token):
    if token.ent_iob != 0 and token.ent_type_ == "PERSON":
        return "[REDACTED] "
    else:
        return token.string

# Loop through all the entities in a document and check if they are names
def scrub(text):
    doc = nlp(text)
    for ent in doc.ents:
        ent.merge()
    tokens = map(replace_name_with_placeholder, doc)
    return "".join(tokens)

s = """
In 1950, Alan Turing published his famous article "Computing Machinery and Intelligence". In 1957, Noam Chomsky’s 
Syntactic Structures revolutionized Linguistics with 'universal grammar', a rule based system of syntactic structures.
"""

print(scrub(s))


In 1950, [REDACTED] published his famous article "Computing Machinery and Intelligence". In 1957, [REDACTED] 
Syntactic Structures revolutionized Linguistics with 'universal grammar', a rule based system of syntactic structures.



## Extracting Facts

In [6]:
import spacy
import textacy.extract

# Load the large English NLP model
nlp = spacy.load('en_core_web_lg')

# The text we want to examine
text = """London is the capital and most populous city of England and  the United Kingdom.  
Standing on the River Thames in the south east of the island of Great Britain, 
London has been a major settlement  for two millennia.  It was founded by the Romans, 
who named it Londinium.
"""

# Parse the document with spaCy
doc = nlp(text)

# Extract semi-structured statements
statements = textacy.extract.semistructured_statements(doc, "London")

# Print the results
print("Here are the things I know about London:")

for statement in statements:
    subject, verb, fact = statement
    print(f" - {fact}")

Here are the things I know about London:
 - the capital and most populous city of England and  the United Kingdom.  

 - a major settlement  for two millennia.  


Here’s one way to extract frequently-mentioned noun chunks from a document:

In [9]:
import spacy
import textacy.extract

# Load the large English NLP model
nlp = spacy.load('en_core_web_lg')

# The text we want to examine
text = """London (/ˈlʌndən/ (About this soundlisten) LUN-dən) is the capital and largest city of the United Kingdom.[9][10] Standing on the River Thames in southeastern England, at the head of its 50-mile (80 km) estuary leading to the North Sea, London has been a major settlement for two millennia. Londinium was founded by the Romans.[11] The City of London, London's ancient core − an area of just 1.12 square miles (2.9 km2) and colloquially known as the Square Mile − retains its medieval boundaries.[12][13][14][15][16][note 1] The City of Westminster is also an Inner London borough holding city status. Greater London is governed by the Mayor of London and the London Assembly.[17][note 2][18] London is often considered as the world's leading global city[19][20][21] and has been termed as the world's most powerful,[22] most desirable,[23] most influential,[24] most visited,[25] most expensive,[26][27] innovative,[28] sustainable,[29] most investment friendly,[30] most popular for work,[31] and the most vegetarian friendly[32] city in the world. London exerts a considerable impact upon the arts, commerce, education, entertainment, fashion, finance, healthcare, media, professional services, research and development, tourism and transportation.[33][34] London ranks 26 out of 300 major cities for economic performance.[35] It is one of the largest financial centres[36] and has either the fifth or sixth largest metropolitan area GDP.[note 3][37][38][39][40][41] It is the most-visited city as measured by international arrivals[42] and has the busiest city airport system as measured by passenger traffic.[43] It is the leading investment destination,[44][45][46][47] hosting more international retailers[48][49] and ultra high-net-worth individuals[50][51] than any other city. London's universities form the largest concentration of higher education institutes in Europe.[52] In 2012, London became the first city to have hosted three modern Summer Olympic Games.[53] London has a diverse range of people and cultures, and more than 300 languages are spoken in the region.[54] Its estimated mid-2016 municipal population (corresponding to Greater London) was 8,787,892,[4] the most populous of any city in the European Union[55] and accounting for 13.4% of the UK population.[56] London's urban area is the second most populous in the EU, after Paris, with 9,787,426 inhabitants at the 2011 census.[57] The population within the London commuter belt is the most populous in the EU with 14,040,163 inhabitants in 2016.[note 4][3][58] London was the world's most populous city from c. 1831 to 1925.[59] London contains four World Heritage Sites: the Tower of London; Kew Gardens; the site comprising the Palace of Westminster, Westminster Abbey, and St Margaret's Church; and the historic settlement in Greenwich where the Royal Observatory, Greenwich defines the Prime Meridian, 0° longitude, and Greenwich Mean Time.[60] Other landmarks include Buckingham Palace, the London Eye, Piccadilly Circus, St Paul's Cathedral, Tower Bridge, Trafalgar Square and The Shard. London has numerous museums, galleries, libraries and sporting events. These include the British Museum, National Gallery, Natural History Museum, Tate Modern, British Library and West End theatres.[61] The London Underground is the oldest underground railway network in the world. Contents 1	Toponymy 2	History 2.1	Prehistory 2.2	Roman London 2.3	Anglo-Saxon and Viking period London 2.4	Middle Ages 2.5	Early modern 2.6	Late modern and contemporary 3	Administration 3.1	Local government 3.2	National government 3.3	Policing and crime 4	Geography 4.1	Scope 4.2	Status 4.3	Topography 4.4	Climate 4.5	Districts 4.6	Architecture 4.7	Cityscape 4.8	Natural history 5	Demography 5.1	Ethnic groups 5.2	Religion 5.3	Accent 6	Economy 6.1	The City of London 6.2	Media and technology 6.3	Tourism 7	Transport 7.1	Aviation 7.2	Rail 7.3	Buses and trams 7.4	Cable car 7.5	Cycling 7.6	Port and river boats 7.7	Roads 8	Education 8.1	Tertiary education 8.2	Primary and secondary education 9	Culture 9.1	Leisure and entertainment 9.2	Literature, film and television 9.3	Museums and art galleries 9.4	Music 10	Notable people 11	Recreation 11.1	Parks and open spaces 11.2	Walking 12	Sport 13	See also 14	Notes 15	References 15.1	Bibliography 16	External links Toponymy Main article: Etymology of London It is an ancient name, attested already in the first century AD, usually in the Latinised form Londinium;[62] for example, handwritten Roman tablets recovered in the city originating from AD 65/70-80 include the word Londinio ("in London").[63] Over the years, the name has attracted many mythicising explanations. The earliest attested appears in Geoffrey of Monmouth's Historia Regum Britanniae, written around 1136.[62] This had it that the name originated from a supposed King Lud, who had allegedly taken over the city and named it Kaerlud.[64] Modern scientific analyses of the name must account for the origins of the different forms found in early sources Latin (usually Londinium), Old English (usually Lunden), and Welsh (usually Llundein), with reference to the known developments over time of sounds in those different languages. It is agreed that the name came into these languages from Common Brythonic; recent work tends to reconstruct the lost Celtic form of the name as *[Londonjon] or something similar. This was adapted into Latin as Londinium and borrowed into Old English, the ancestor-language of English.[65] The toponymy of the Common Brythonic form is much debated. A prominent explanation was Richard Coates's 1998 argument that the name derived from pre-Celtic Old European *(p)lowonida, meaning "river too wide to ford". Coates suggested that this was a name given to the part of the River Thames which flows through London; from this, the settlement gained the Celtic form of its name, *Lowonidonjon.[66] However, most work has accepted a Celtic origin for the name, and recent studies have favoured an explanation along the lines of a Celtic derivative of an proto-Indo-European root *lendh- ('sink, cause to sink'), combined with the Celtic suffix *-injo- or *-onjo- (used to form place-names). Peter Schrijver has specifically suggested, on these grounds, that the name originally meant 'place that floods (periodically, tidally)'.[67][65] Until 1889, the name "London" applied to the City of London, but since then it has also referred to the County of London and Greater London.[68] "London" is sometimes written informally as "LDN".[69] History Main articles: History of London and Timeline of London Prehistory In 1993, the remains of a Bronze Age bridge were found on the south foreshore, upstream of Vauxhall Bridge.[70] This bridge either crossed the Thames or reached a now lost island in it. Two of those timbers were radiocarbon dated to between 1750 BC and 1285 BC.[70] In 2010 the foundations of a large timber structure, dated to between 4800 BC and 4500 BC,[71] were found on the Thames's south foreshore, downstream of Vauxhall Bridge.[72] The function of the mesolithic structure is not known. Both structures are on the south bank where the River Effra flows into the Thames.[72] Roman London In 1300, the City was still confined within the Roman walls Although there is evidence of scattered Brythonic settlements in the area, the first major settlement was founded by the Romans about four years[1] after the invasion of AD 43.[73] This lasted only until around AD 61, when the Iceni tribe led by Queen Boudica stormed it, burning it to the ground.[74] The next, heavily planned, incarnation of Londinium prospered, and it superseded Colchester as the capital of the Roman province of Britannia in 100. At its height in the 2nd century, Roman London had a population of around 60,000.[75] Anglo-Saxon and Viking period London With the collapse of Roman rule in the early 5th century, London ceased to be a capital, and the walled city of Londinium was effectively abandoned, although Roman civilisation continued in the area of St Martin-in-the-Fields until around 450.[76] From around 500, an Anglo-Saxon settlement known as Lundenwic developed slightly west of the old Roman city.[77] By about 680, the city had regrown into a major port, although there is little evidence of large-scale production. From the 820s repeated Viking assaults brought decline. Three are recorded; those in 851 and 886 succeeded, while the last, in 994, was rebuffed.[78] The Lancastrian siege of London in 1471 is attacked by a Yorkist sally The Vikings established Danelaw over much of eastern and northern England; its boundary stretched roughly from London to Chester. It was an area of political and geographical control imposed by the Viking incursions which was formally agreed by the Danish warlord, Guthrum and the West Saxon king Alfred the Great in 886. The Anglo-Saxon Chronicle recorded that Alfred "refounded" London in 886. Archaeological research shows that this involved abandonment of Lundenwic and a revival of life and trade within the old Roman walls. London then grew slowly until about 950, after which activity increased dramatically.[79] By the 11th century, London was beyond all comparison the largest town in England. Westminster Abbey, rebuilt in the Romanesque style by King Edward the Confessor, was one of the grandest churches in Europe. Winchester had previously been the capital of Anglo-Saxon England, but from this time on, London became the main forum for foreign traders and the base for defence in time of war. In the view of Frank Stenton: "It had the resources, and it was rapidly developing the dignity and the political self-consciousness appropriate to a national capital."[80][81] Middle Ages Westminster Abbey, as seen in this painting (by Canaletto, 1749), is a World Heritage Site and one of London's oldest and most important buildings After winning the Battle of Hastings, William, Duke of Normandy was crowned King of England in the newly completed Westminster Abbey on Christmas Day 1066.[82] William constructed the Tower of London, the first of the many Norman castles in England to be rebuilt in stone, in the southeastern corner of the city, to intimidate the native inhabitants.[83] In 1097, William II began the building of Westminster Hall, close by the abbey of the same name. The hall became the basis of a new Palace of Westminster.[84][85] In the 12th century, the institutions of central government, which had hitherto accompanied the royal English court as it moved around the country, grew in size and sophistication and became increasingly fixed in one place. For most purposes this was Westminster, although the royal treasury, having been moved from Winchester, came to rest in the Tower. While the City of Westminster developed into a true capital in governmental terms, its distinct neighbour, the City of London, remained England's largest city and principal commercial centre, and it flourished under its own unique administration, the Corporation of London. In 1100, its population was around 18,000; by 1300 it had grown to nearly 100,000.[86] Disaster struck in the form of the Black Death in the mid-14th century, when London lost nearly a third of its population.[87] London was the focus of the Peasants' Revolt in 1381.[88] London was also a centre of England's Jewish population before their expulsion by Edward I in 1290. Violence against Jews took place in 1190, after it was rumoured that the new King had ordered their massacre after they had presented themselves at his coronation.[89] In 1264 during the Second Barons' War, Simon de Montfort's rebels killed 500 Jews while attempting to seize records of debts.[90] Early modern Map of London in 1593. There is only one bridge across the Thames, but parts of Southwark on the south bank of the river have been developed. During the Tudor period the Reformation produced a gradual shift to Protestantism, and much of London property passed from church to private ownership, which accelerated trade and business in the city.[91] In 1475, the Hanseatic League set up its main trading base (kontor) of England in London, called the Stalhof or Steelyard. It existed until 1853, when the Hanseatic cities of Lübeck, Bremen and Hamburg sold the property to South Eastern Railway.[92] Woollen cloth was shipped undyed and undressed from 14th/15th century London to the nearby shores of the Low Countries, where it was considered indispensable.[93] But the reach of English maritime enterprise hardly extended beyond the seas of north-west Europe. The commercial route to Italy and the Mediterranean Sea normally lay through Antwerp and over the Alps; any ships passing through the Strait of Gibraltar to or from England were likely to be Italian or Ragusan. Upon the re-opening of the Netherlands to English shipping in January 1565, there ensued a strong outburst of commercial activity.[94] The Royal Exchange was founded.[95] Mercantilism grew, and monopoly trading companies such as the East India Company were established, with trade expanding to the New World. London became the principal North Sea port, with migrants arriving from England and abroad. The population rose from an estimated 50,000 in 1530 to about 225,000 in 1605.[91] In the 16th century William Shakespeare and his contemporaries lived in London at a time of hostility to the development of the theatre. By the end of the Tudor period in 1603, London was still very compact. There was an assassination attempt on James I in Westminster, in the Gunpowder Plot on 5 November 1605.[96] Vertue's 1738 plan of the Lines of Communication, built during the English Civil War In the English Civil War the majority of Londoners supported the Parliamentary cause. After an initial advance by the Royalists in 1642, culminating in the battles of Brentford and Turnham Green, London was surrounded by a defensive perimeter wall known as the Lines of Communication. The lines were built by up to 20,000 people, and were completed in under two months.[97] The fortifications failed their only test when the New Model Army entered London in 1647,[98] and they were levelled by Parliament the same year.[99] London was plagued by disease in the early 17th century,[100] culminating in the Great Plague of 1665–1666, which killed up to 100,000 people, or a fifth of the population.[101] The Great Fire of London destroyed many parts of the city in 1666 The Great Fire of London broke out in 1666 in Pudding Lane in the city and quickly swept through the wooden buildings.[102] Rebuilding took over ten years and was supervised by Robert Hooke[103][104][105] as Surveyor of London.[106] In 1708 Christopher Wren's masterpiece, St Paul's Cathedral was completed. During the Georgian era, new districts such as Mayfair were formed in the west; new bridges over the Thames encouraged development in South London. In the east, the Port of London expanded downstream. London's development as an international financial centre matured for much of the 1700s. In 1762, George III acquired Buckingham House and it was enlarged over the next 75 years. During the 18th century, London was dogged by crime, and the Bow Street Runners were established in 1750 as a professional police force.[107] In total, more than 200 offences were punishable by death,[108] including petty theft.[109] Most children born in the city died before reaching their third birthday.[110] View to the Royal Exchange in the City of London in 1886 The coffeehouse became a popular place to debate ideas, with growing literacy and the development of the printing press making news widely available; and Fleet Street became the centre of the British press. Following the invasion of Amsterdam by Napoleonic armies, many financiers relocated to London, especially a large Jewish community, and the first London international issue[clarification needed] was arranged in 1817. Around the same time, the Royal Navy became the world leading war fleet, acting as a serious deterrent to potential economic adversaries of the United Kingdom. The repeal of the Corn Laws in 1846 was specifically aimed at weakening Dutch economic power. London then overtook Amsterdam as the leading international financial centre.[111] In 1888, London became home to a series of murders by a man known only as Jack the Ripper and It has since become one of the world's most famous unsolved mysteries. According to Samuel Johnson: You find no man, at all intellectual, who is willing to leave London. No, Sir, when a man is tired of London, he is tired of life; for there is in London all that life can afford. — Samuel Johnson, 1777[112] Late modern and contemporary British volunteer recruits in London, August 1914 London was the world's largest city from c.1831 to 1925.[59] London's overcrowded conditions led to cholera epidemics,[113] claiming 14,000 lives in 1848, and 6,000 in 1866.[114] Rising traffic congestion led to the creation of the world's first local urban rail network. The Metropolitan Board of Works oversaw infrastructure expansion in the capital and some of the surrounding counties; it was abolished in 1889 when the London County Council was created out of those areas of the counties surrounding the capital. London was bombed by the Germans during the First World War,[115] and during the Second World War, the Blitz and other bombings by the German Luftwaffe killed over 30,000 Londoners, destroying large tracts of housing and other buildings across the city.[116] Immediately after the war, the 1948 Summer Olympics were held at the original Wembley Stadium, at a time when London was still recovering from the war.[117] A bombed-out London street during the Blitz of the Second World War From the 1940s onwards, London became home to a large number of immigrants, primarily from Commonwealth countries such as Jamaica, India, Bangladesh and Pakistan,[118] making London one of the most diverse cities worldwide. In 1951, the Festival of Britain was held on the South Bank.[119] The Great Smog of 1952 led to the Clean Air Act 1956, which ended the "pea soup fogs" for which London had been notorious.[120] Primarily starting in the mid-1960s, London became a centre for the worldwide youth culture, exemplified by the Swinging London subculture[121] associated with the King's Road, Chelsea[122] and Carnaby Street.[123] The role of trendsetter was revived during the punk era.[124] In 1965 London's political boundaries were expanded to take into account the growth of the urban area and a new Greater London Council was created.[125] During The Troubles in Northern Ireland, London was subjected to bombing attacks by the Provisional IRA[126] for two decades, starting with the Old Bailey bombing in 1973.[127][128] Racial inequality was highlighted by the 1981 Brixton riot.[129] Greater London's population declined steadily in the decades after the Second World War, from an estimated peak of 8.6 million in 1939 to around 6.8 million in the 1980s.[130] The principal ports for London moved downstream to Felixstowe and Tilbury, with the London Docklands area becoming a focus for regeneration, including the Canary Wharf development. This was borne out of London's ever-increasing role as a major international financial centre during the 1980s.[131] The Thames Barrier was completed in the 1980s to protect London against tidal surges from the North Sea.[132] The Greater London Council was abolished in 1986, which left London without a central administration until 2000 when London-wide government was restored, with the creation of the Greater London Authority.[133] To celebrate the start of the 21st century, the Millennium Dome, London Eye and Millennium Bridge were constructed.[134] On 6 July 2005 London was awarded the 2012 Summer Olympics, making London the first city to stage the Olympic Games three times.[135] On 7 July 2005, three London Underground trains and a double-decker bus were bombed in a series of terrorist attacks.[136] In 2008, London named alongside New York City and Hong Kong as Nylonkong, being hailed as the world's three most influential global cities.[137] In January 2015, Greater London's population was estimated to be 8.63 million, the highest level since 1939.[138] During the Brexit referendum in 2016, the UK as a whole decided to leave the European Union, but a majority of London constituencies voted to remain in the EU.[139] Administration London City hall London at dawn (cropped).jpg This article is part of a series on the politics and government of London Greater London Authority[show] London Councils[show] London in the UK and EU[show] Royal Coat of Arms of the United Kingdom.svg British politics portal vte Local government Main articles: Local government in London, History of local government in London, and List of heads of London government The administration of London is formed of two tiers: a citywide, strategic tier and a local tier. Citywide administration is coordinated by the Greater London Authority (GLA), while local administration is carried out by 33 smaller authorities.[140] The GLA consists of two elected components: the Mayor of London, who has executive powers, and the London Assembly, which scrutinises the mayor's decisions and can accept or reject the mayor's budget proposals each year. The headquarters of the GLA is City Hall, Southwark; the mayor is Sadiq Khan, the first Muslim mayor of a major Western capital.[141][142] The mayor's statutory planning strategy is published as the London Plan, which was most recently revised in 2011.[143] The local authorities are the councils of the 32 London boroughs and the City of London Corporation.[144] They are responsible for most local services, such as local planning, schools, social services, local roads and refuse collection. Certain functions, such as waste management, are provided through joint arrangements. In 2009–2010 the combined revenue expenditure by London councils and the GLA amounted to just over £22 billion (£14.7 billion for the boroughs and £7.4 billion for the GLA).[145] The London Fire Brigade is the statutory fire and rescue service for Greater London. It is run by the London Fire and Emergency Planning Authority and is the third largest fire service in the world.[146] National Health Service ambulance services are provided by the London Ambulance Service (LAS) NHS Trust, the largest free-at-the-point-of-use emergency ambulance service in the world.[147] The London Air Ambulance charity operates in conjunction with the LAS where required. Her Majesty's Coastguard and the Royal National Lifeboat Institution operate on the River Thames,[148][149] which is under the jurisdiction of the Port of London Authority from Teddington Lock to the sea.[150] National government 10 Downing Street, official residence of the Prime Minister London is the seat of the Government of the United Kingdom. Many government departments, as well as the Prime Minister's residence at 10 Downing Street, are based close to the Palace of Westminster, particularly along Whitehall.[151] The British Parliament is often referred to as the "Mother of Parliaments" (although this sobriquet was first applied to England itself, and not to Westminster, by John Bright)[152] because it has been a model for many other parliamentary systems.[152] There are 73 Members of Parliament (MPs) from London, elected from local parliamentary constituencies in the national Parliament. As of May 2015, 49 are from the Labour Party, 21 are Conservatives, and three are Liberal Democrat.[153] The UK government ministerial post of Minister for London was created in 1994 and currently occupied by Jo Johnson.[154] Policing and crime Main article: Crime in London Policing in Greater London, with the exception of the City of London, is provided by the Metropolitan Police Service, overseen by the Mayor through the Mayor's Office for Policing and Crime (MOPAC).[155][156] The City of London has its own police force – the City of London Police.[157] The British Transport Police are responsible for police services on National Rail, London Underground, Docklands Light Railway and Tramlink services.[158] A fourth police force in London, the Ministry of Defence Police, do not generally become involved with policing the general public. Crime rates vary widely by area, ranging from parts with serious issues to parts considered very safe. Today crime figures are made available nationally at Local Authority[159] and Ward level.[160] In 2015 there were 118 homicides, a 25.5% increase over 2014.[161] The Metropolitan Police have made detailed crime figures, broken down by category at borough and ward level, available on their website since 2000.[162] Recorded crime has been rising in London, notably violent crime and murder by stabbing and other means have risen. There have been 50 murders from the start of 2018 to mid April 2018. Funding cuts to police in London are likely to have contributed to this, though other factors are also involved.[163] Geography Main article: Geography of London Scope Satellite view of inner London (2010) London, also referred to as Greater London, is one of nine regions of England and the top-level subdivision covering most of the city's metropolis.[note 5] The small ancient City of London at its core once comprised the whole settlement, but as its urban area grew, the Corporation of London resisted attempts to amalgamate the city with its suburbs, causing "London" to be defined in a number of ways for different purposes.[164] Forty per cent of Greater London is covered by the London post town, within which 'LONDON' forms part of postal addresses.[165][166] The London telephone area code (020) covers a larger area, similar in size to Greater London, although some outer districts are excluded and some places just outside are included. The Greater London boundary has been aligned to the M25 motorway in places.[167] Outward urban expansion is now prevented by the Metropolitan Green Belt,[168] although the built-up area extends beyond the boundary in places, resulting in a separately defined Greater London Urban Area. Beyond this is the vast London commuter belt.[169] Greater London is split for some purposes into Inner London and Outer London.[170] The city is split by the River Thames into North and South, with an informal central London area in its interior. The coordinates of the nominal centre of London, traditionally considered to be the original Eleanor Cross at Charing Cross near the junction of Trafalgar Square and Whitehall, are about 51°30′26″N 00°07′39″W.[171] However the geographical centre of London, on one definition, is in the London Borough of Lambeth, just 0.1 miles to the northeast of Lambeth North tube station.[172] Status Within London, both the City of London and the City of Westminster have city status and both the City of London and the remainder of Greater London are counties for the purposes of lieutenancies.[173] The area of Greater London has incorporated areas that are part of the historic counties of Middlesex, Kent, Surrey, Essex and Hertfordshire.[174] London's status as the capital of England, and later the United Kingdom, has never been granted or confirmed officially—by statute or in written form.[note 6] Its position was formed through constitutional convention, making its status as de facto capital a part of the UK's uncodified constitution. The capital of England was moved to London from Winchester as the Palace of Westminster developed in the 12th and 13th centuries to become the permanent location of the royal court, and thus the political capital of the nation.[178] More recently, Greater London has been defined as a region of England and in this context is known as London.[15] Topography London from Primrose Hill Greater London encompasses a total area of 1,583 square kilometres (611 sq mi), an area which had a population of 7,172,036 in 2001 and a population density of 4,542 inhabitants per square kilometre (11,760/sq mi). The extended area known as the London Metropolitan Region or the London Metropolitan Agglomeration, comprises a total area of 8,382 square kilometres (3,236 sq mi) has a population of 13,709,000 and a population density of 1,510 inhabitants per square kilometre (3,900/sq mi).[179] Modern London stands on the Thames, its primary geographical feature, a navigable river which crosses the city from the south-west to the east. The Thames Valley is a floodplain surrounded by gently rolling hills including Parliament Hill, Addington Hills, and Primrose Hill. Historically London grew up at the lowest bridging point on the Thames. The Thames was once a much broader, shallower river with extensive marshlands; at high tide, its shores reached five times their present width.[180] Since the Victorian era the Thames has been extensively embanked, and many of its London tributaries now flow underground. The Thames is a tidal river, and London is vulnerable to flooding.[181] The threat has increased over time because of a slow but continuous rise in high water level by the slow 'tilting' of the British Isles (up in Scotland and Northern Ireland and down in southern parts of England, Wales and Ireland) caused by post-glacial rebound.[182][183] In 1974, a decade of work began on the construction of the Thames Barrier across the Thames at Woolwich to deal with this threat. While the barrier is expected to function as designed until roughly 2070, concepts for its future enlargement or redesign are already being discussed.[184] Climate Main article: Climate of London London has a temperate oceanic climate (Köppen: Cfb ) receiving less precipitation than Rome, Bordeaux, Lisbon, Naples, Sydney and New York City.[185][186][187][188][189][190] Temperature extremes in London range from 38.1 °C (100.6 °F) at Kew during August 2003[191] down to −16.1 °C (3.0 °F) at Northolt during January 1962.[192] Although, a temperature of −21.0 °C (−5.8 °F) was recorded in January 1795. Summers are generally warm, sometimes hot. London's average July high is 24 °C (74 °F). On average London will see 31 days above 25 °C (77.0 °F) each year, and 4.2 days above 30.0 °C (86.0 °F) every year. During the 2003 European heat wave there were 14 consecutive days above 30 °C (86.0 °F) and 2 consecutive days where temperatures reached 38 °C (100 °F), leading to hundreds of heat related deaths.[193] Winters are generally cool with little temperature variation. Heavy Snow is rare but snow happens at least once each winter. Spring and autumn can be pleasant. As a large city, London has a considerable urban heat island effect,[194] making the centre of London at times 5 °C (9 °F) warmer than the suburbs and outskirts. This can be seen below when comparing London Heathrow, 15 miles (24 km) west of London, with the London Weather Centre.[195] Climate data for London (LHR), elevation: 25 m or 82 ft, 1981–2010 normals, extremes 1948–present Month	Jan	Feb	Mar	Apr	May	Jun	Jul	Aug	Sep	Oct	Nov	Dec	Year Record high °C (°F)	17.2 (63)	19.8 (67.6)	24.2 (75.6)	29.4 (84.9)	32.8 (91)	35.6 (96.1)	36.7 (98.1)	38.1 (100.6)	35.4 (95.7)	29.9 (85.8)	20.8 (69.4)	17.4 (63.3)	38.1 (100.6) Mean maximum °C (°F)	13.1 (55.6)	14.1 (57.4)	17.2 (63)	21.3 (70.3)	25.8 (78.4)	28.5 (83.3)	30.1 (86.2)	29.7 (85.5)	25.5 (77.9)	20.5 (68.9)	16.2 (61.2)	13.5 (56.3)	31.5 (88.7) Average high °C (°F)	8.1 (46.6)	8.4 (47.1)	11.3 (52.3)	14.2 (57.6)	17.9 (64.2)	21.0 (69.8)	23.5 (74.3)	23.2 (73.8)	19.9 (67.8)	15.5 (59.9)	11.1 (52)	8.3 (46.9)	15.2 (59.4) Daily mean °C (°F)	5.2 (41.4)	5.3 (41.5)	7.6 (45.7)	9.9 (49.8)	13.3 (55.9)	16.4 (61.5)	18.7 (65.7)	18.5 (65.3)	15.7 (60.3)	12.0 (53.6)	8.0 (46.4)	5.5 (41.9)	11.3 (52.3) Average low °C (°F)	2.3 (36.1)	2.1 (35.8)	3.9 (39)	5.5 (41.9)	8.7 (47.7)	11.7 (53.1)	13.9 (57)	13.7 (56.7)	11.4 (52.5)	8.4 (47.1)	4.9 (40.8)	2.7 (36.9)	7.4 (45.4) Mean minimum °C (°F)	−4.2 (24.4)	−3.9 (25)	−2.0 (28.4)	−0.3 (31.5)	3.1 (37.6)	6.7 (44.1)	9.5 (49.1)	9.1 (48.4)	6.0 (42.8)	2.0 (35.6)	−1.7 (28.9)	−4.1 (24.6)	−6.1 (21) Record low °C (°F)	−13.2 (8.2)	−9.6 (14.7)	−5.1 (22.8)	−2.6 (27.3)	−0.9 (30.4)	1.5 (34.7)	5.6 (42.1)	5.9 (42.6)	1.8 (35.2)	−3.3 (26.1)	−7.0 (19.4)	−11.8 (10.8)	−13.2 (8.2) Average precipitation mm (inches)	55.2 (2.17)	40.9 (1.61)	41.6 (1.64)	43.7 (1.72)	49.4 (1.94)	45.1 (1.78)	44.5 (1.75)	49.5 (1.95)	49.1 (1.93)	68.5 (2.7)	59.0 (2.32)	55.2 (2.17)	601.7 (23.68) Average precipitation days (≥ 1.0 mm)	11.1	8.5	9.3	9.1	8.8	8.2	7.7	7.5	8.1	10.8	10.3	10.2	109.6 Mean monthly sunshine hours	61.5	77.9	114.6	168.7	198.5	204.3	212.0	204.7	149.3	116.5	72.6	52.0	1,632.6 Source: Met Office [196] Royal Netherlands Meteorological Institute [197][198] For more station data near London, see Geography of London. Climate data for London Weather Centre, 2001–2014 normals Districts Main articles: List of districts of London and London boroughs London's vast urban area is often described using a set of district names, such as Bloomsbury, Mayfair, Wembley and Whitechapel. These are either informal designations, reflect the names of villages that have been absorbed by sprawl, or are superseded administrative units such as parishes or former boroughs. Such names have remained in use through tradition, each referring to a local area with its own distinctive character, but without official boundaries. Since 1965 Greater London has been divided into 32 London boroughs in addition to the ancient City of London.[201][202] The City of London is the main financial district,[203] and Canary Wharf has recently developed into a new financial and commercial hub in the Docklands to the east. The West End is London's main entertainment and shopping district, attracting tourists.[204] West London includes expensive residential areas where properties can sell for tens of millions of pounds.[205] The average price for properties in Kensington and Chelsea is over £2 million with a similarly high outlay in most of central London.[206][207] The East End is the area closest to the original Port of London, known for its high immigrant population, as well as for being one of the poorest areas in London.[208] The surrounding East London area saw much of London's early industrial development; now, brownfield sites throughout the area are being redeveloped as part of the Thames Gateway including the London Riverside and Lower Lea Valley, which was developed into the Olympic Park for the 2012 Olympics and Paralympics.[208] Architecture Main articles: Architecture of London, List of tallest buildings and structures in London, and List of demolished buildings and structures in London The Tower of London is a historic medieval castle, the oldest part of which dates back to 1078 London's buildings are too diverse to be characterised by any particular architectural style, partly because of their varying ages. Many grand houses and public buildings, such as the National Gallery, are constructed from Portland stone. Some areas of the city, particularly those just west of the centre, are characterised by white stucco or whitewashed buildings. Few structures in central London pre-date the Great Fire of 1666, these being a few trace Roman remains, the Tower of London and a few scattered Tudor survivors in the City. Further out is, for example, the Tudor-period Hampton Court Palace, England's oldest surviving Tudor palace, built by Cardinal Thomas Wolsey c.1515.[209] 17th-century churches by Wren, neoclassical financial institutions such as the Royal Exchange and the Bank of England, to the early 20th century Old Bailey and the 1960s Barbican Estate form part of the varied architectural heritage. 30 St Mary Axe, also known as "the Gherkin", towers over St Andrew Undershaft; modern architecture juxtaposed by historic architecture is seen often in London Trafalgar Square and its fountains, with Nelson's Column on the right The disused – but soon to be rejuvenated – 1939 Battersea Power Station by the river in the south-west is a local landmark, while some railway termini are excellent examples of Victorian architecture, most notably St. Pancras and Paddington.[210] The density of London varies, with high employment density in the central area, high residential densities in inner London, and lower densities in Outer London. The Monument in the City of London provides views of the surrounding area while commemorating the Great Fire of London, which originated nearby. Marble Arch and Wellington Arch, at the north and south ends of Park Lane, respectively, have royal connections, as do the Albert Memorial and Royal Albert Hall in Kensington. Nelson's Column is a nationally recognised monument in Trafalgar Square, one of the focal points of central London. Older buildings are mainly brick built, most commonly the yellow London stock brick or a warm orange-red variety, often decorated with carvings and white plaster mouldings.[211] In the dense areas, most of the concentration is via medium- and high-rise buildings. London's skyscrapers, such as 30 St Mary Axe, Tower 42, the Broadgate Tower and One Canada Square, are mostly in the two financial districts, the City of London and Canary Wharf. High-rise development is restricted at certain sites if it would obstruct protected views of St Paul's Cathedral and other historic buildings. Nevertheless, there are a number of very tall skyscrapers in central London (see Tall buildings in London), including the 95-storey Shard London Bridge, the tallest building in the European Union. Other notable modern buildings include City Hall in Southwark with its distinctive oval shape[212] and the British Library in Somers Town/Kings Cross. What was formerly the Millennium Dome, by the Thames to the east of Canary Wharf, is now an entertainment venue called the O2 Arena. Cityscape The Houses of Parliament and Elizabeth Tower on the right foreground, the London Eye on the left foreground and The Shard with Canary Wharf in the background; seen in September 2014 Tower Bridge and The Shard in April 2017 Supermoon over the City of London seen from Tate Modern in January 2018 The Royal Greenwich Observatory with Canary Wharf in the background in October 2016 Natural history The London Natural History Society suggest that London is "one of the World's Greenest Cities" with more than 40 per cent green space or open water. They indicate that 2000 species of flowering plant have been found growing there and that the tidal Thames supports 120 species of fish.[213] They also state that over 60 species of bird nest in central London and that their members have recorded 47 species of butterfly, 1173 moths and more than 270 kinds of spider around London. London's wetland areas support nationally important populations of many water birds. London has 38 Sites of Special Scientific Interest (SSSIs), two National Nature Reserves and 76 Local Nature Reserves.[214] Amphibians are common in the capital, including smooth newts living by the Tate Modern, and common frogs, common toads, palmate newts and great crested newts. On the other hand, native reptiles such as slowworms, common lizards, grass snakes and adders, are mostly only seen in Outer London.[215] Fox on Ayres Street, Southwark, South London Among other inhabitants of London are 10,000 red foxes, so that there are now 16 foxes for every square mile (2.6 square kilometres) of London. These urban foxes are noticeably bolder than their country cousins, sharing the pavement with pedestrians and raising cubs in people's backyards. Foxes have even sneaked into the Houses of Parliament, where one was found asleep on a filing cabinet. Another broke into the grounds of Buckingham Palace, reportedly killing some of Queen Elizabeth II's prized pink flamingos. Generally, however, foxes and city folk appear to get along. A survey in 2001 by the London-based Mammal Society found that 80 per cent of 3,779 respondents who volunteered to keep a diary of garden mammal visits liked having them around. This sample cannot be taken to represent Londoners as a whole.[216][217] Other mammals found in Greater London are hedgehogs, rats, mice, rabbit, shrew, vole, and squirrels,[218] In wilder areas of Outer London, such as Epping Forest, a wide variety of mammals are found including hare, badger, field, bank and water vole, wood mouse, yellow-necked mouse, mole, shrew, and weasel, in addition to fox, squirrel and hedgehog. A dead otter was found at The Highway, in Wapping, about a mile from the Tower Bridge, which would suggest that they have begun to move back after being absent a hundred years from the city.[219] Ten of England's eighteen species of bats have been recorded in Epping Forest: soprano, nathusius and common pipistrelles, noctule, serotine, barbastelle, daubenton's, brown Long-eared, natterer's and leisler's.[220] Among the strange sights seen in London have been a whale in the Thames,[221] while the BBC Two programme "Natural World: Unnatural History of London" shows pigeons using the London Underground to get around the city, a seal that takes fish from fishmongers outside Billingsgate Fish Market, and foxes that will "sit" if given sausages.[222] Herds of red and fallow deer also roam freely within much of Richmond and Bushy Park. A cull takes place each November and February to ensure numbers can be sustained.[223] Epping Forest is also known for its fallow deer, which can frequently be seen in herds to the north of the Forest. A rare population of melanistic, black fallow deer is also maintained at the Deer Sanctuary near Theydon Bois. Muntjac deer, which escaped from deer parks at the turn of the twentieth century, are also found in the forest. While Londoners are accustomed to wildlife such as birds and foxes sharing the city, more recently urban deer have started becoming a regular feature, and whole herds of fallow deer come into residential areas at night to take advantage of London's green spaces.[224][225] Demography Main article: Demography of London 2011 United Kingdom Census[226] Country of birth	Population United Kingdom United Kingdom	5,175,677 India India	262,247 Poland Poland	158,300 Republic of Ireland Ireland	129,807 Nigeria Nigeria	114,718 Pakistan Pakistan	112,457 Bangladesh Bangladesh	109,948 Jamaica Jamaica	87,467 Sri Lanka Sri Lanka	84,542 France France	66,654 London maps showing the percentage distribution of selected races according to the 2011 Census White White British Asian Asian British Black Black British The 2011 census recorded that 2,998,264 people or 36.7% of London's population are foreign-born making London the city with the second largest immigrant population, behind New York City, in terms of absolute numbers. About 69% of children born in London in 2015 had at least one parent who was born abroad.[227] The table to the right shows the most common countries of birth of London residents. Note that some of the German-born population, in 18th position, are British citizens from birth born to parents serving in the British Armed Forces in Germany.[228] With increasing industrialisation, London's population grew rapidly throughout the 19th and early 20th centuries, and it was for some time in the late 19th and early 20th centuries the most populous city in the world. Its population peaked at 8,615,245 in 1939 immediately before the outbreak of the Second World War, but had declined to 7,192,091 at the 2001 Census. However, the population then grew by just over a million between the 2001 and 2011 Censuses, to reach 8,173,941 in the latter enumeration.[229] However, London's continuous urban area extends beyond the borders of Greater London and was home to 9,787,426 people in 2011,[57] while its wider metropolitan area has a population of between 12 and 14 million depending on the definition used.[230][231] According to Eurostat, London is the most populous city and metropolitan area of the European Union and the second most populous in Europe. During the period 1991–2001 a net 726,000 immigrants arrived in London.[232] The region covers an area of 1,579 square kilometres (610 sq mi). The population density is 5,177 inhabitants per square kilometre (13,410/sq mi),[233] more than ten times that of any other British region.[234] In terms of population, London is the 19th largest city and the 18th largest metropolitan region.[235][236] Ethnic groups Main article: Ethnic groups in London Ethnic groups in the 2011 census [237] White British (44.9%) Other White (14.9%) Asian British (18.4%) Black British (13.3%) Arab British (1.3%) Mixed (5%) Other (2.2%) According to the Office for National Statistics, based on the 2011 Census estimates, 59.8 per cent of the 8,173,941 inhabitants of London were White, with 44.9 per cent White British, 2.2 per cent White Irish, 0.1 per cent gypsy/Irish traveller and 12.1 per cent classified as Other White. 20.9 per cent of Londoners are of Asian and mixed-Asian descent. 19.7 per cent are of full Asian descent, with those of mixed-Asian heritage comprising 1.2 of the population. Indians account for 6.6 per cent of the population, followed by Pakistanis and Bangladeshis at 2.7 per cent each. Chinese peoples account for 1.5 per cent of the population, with Arabs comprising 1.3 per cent. A further 4.9 per cent are classified as "Other Asian". 15.6 per cent of London's population are of Black and mixed-Black descent. 13.3 per cent are of full Black descent, with those of mixed-Black heritage comprising 2.3 per cent. Black Africans account for 7.0 per cent of London's population, with 4.2 per cent as Black Caribbean and 2.1 per cent as "Other Black". 5.0 per cent are of mixed race. Across London, Black and Asian children outnumber White British children by about six to four in state schools.[238] Altogether at the 2011 census, of London's 1,624,768 population aged 0 to 15, 46.4 per cent were White, 19.8 per cent were Asian, 19 per cent were Black, 10.8 per cent were Mixed and 4 per cent represented another ethnic group.[239] In January 2005, a survey of London's ethnic and religious diversity claimed that there were more than 300 languages spoken in London and more than 50 non-indigenous communities with a population of more than 10,000.[240] Figures from the Office for National Statistics show that, in 2010, London's foreign-born population was 2,650,000 (33 per cent), up from 1,630,000 in 1997. The 2011 census showed that 36.7 per cent of Greater London's population were born outside the UK.[241] A portion of the German-born population are likely to be British nationals born to parents serving in the British Armed Forces in Germany.[242] Estimates produced by the Office for National Statistics indicate that the five largest foreign-born groups living in London in the period July 2009 to June 2010 were those born in India, Poland, the Republic of Ireland, Bangladesh and Nigeria.[243] Religion Main article: Religion in London Religion in London (2011 census)[244] Religion			Percent(%)	 Christian	   48.4% No religion	   20.7% Muslim	   12.4% Undeclared	   8.5% Hindu	   5.0% Jewish	   1.8% Sikh	   1.5% Buddhist	   1.0% Other	   0.6% According to the 2011 Census, the largest religious groupings are Christians (48.4 per cent), followed by those of no religion (20.7 per cent), Muslims (12.4 per cent), no response (8.5 per cent), Hindus (5.0 per cent), Jews (1.8 per cent), Sikhs (1.5 per cent), Buddhists (1.0 per cent) and other (0.6 per cent). London has traditionally been Christian, and has a large number of churches, particularly in the City of London. The well-known St Paul's Cathedral in the City and Southwark Cathedral south of the river are Anglican administrative centres,[245] while the Archbishop of Canterbury, principal bishop of the Church of England and worldwide Anglican Communion, has his main residence at Lambeth Palace in the London Borough of Lambeth.[246] St Paul's Cathedral, the seat of the Bishop of London. The BAPS Shri Swaminarayan Mandir London is the second-largest Hindu temple in England and Europe. Important national and royal ceremonies are shared between St Paul's and Westminster Abbey.[247] The Abbey is not to be confused with nearby Westminster Cathedral, which is the largest Roman Catholic cathedral in England and Wales.[248] Despite the prevalence of Anglican churches, observance is very low within the Anglican denomination. Church attendance continues on a long, slow, steady decline, according to Church of England statistics.[249] London is also home to sizeable Muslim, Hindu, Sikh, and Jewish communities. Notable mosques include the East London Mosque in Tower Hamlets, which is allowed to give the Islamic call to prayer through loudspeakers, the London Central Mosque on the edge of Regent's Park[250] and the Baitul Futuh of the Ahmadiyya Muslim Community. Following the oil boom, increasing numbers of wealthy Middle-Eastern Arab Muslims have based themselves around Mayfair, Kensington, and Knightsbridge in West London.[251][252][253] There are large Muslim communities in the eastern boroughs of Tower Hamlets and Newham.[254] Large Hindu communities are in the north-western boroughs of Harrow and Brent, the latter of which hosts what was, until 2006,[255] Europe's largest Hindu temple, Neasden Temple.[256] London is also home to 44 Hindu temples, including the BAPS Shri Swaminarayan Mandir London. There are Sikh communities in East and West London, particularly in Southall, home to one of the largest Sikh populations and the largest Sikh temple outside India.[257] The majority of British Jews live in London, with significant Jewish communities in Stamford Hill, Stanmore, Golders Green, Finchley, Hampstead, Hendon and Edgware in North London. Bevis Marks Synagogue in the City of London is affiliated to London's historic Sephardic Jewish community. It is the only synagogue in Europe which has held regular services continuously for over 300 years. Stanmore and Canons Park Synagogue has the largest membership of any single Orthodox synagogue in the whole of Europe, overtaking Ilford synagogue (also in London) in 1998.[258] The community set up the London Jewish Forum in 2006 in response to the growing significance of devolved London Government.[259] Accent There are many accents traditionally associated with London. The most well known of the London accents long ago acquired the Cockney label from London's East End, which is heard both in London itself, and across the wider South East England region more generally.[260] The accent of a 21st-century Londoner varies widely; what is becoming more and more common amongst the under-30s however is some fusion of Cockney with a whole array of ethnic accents, in particular Caribbean, which form an accent labelled Multicultural London English (MLE).[261] The other widely heard and spoken accent is RP (Received Pronunciation) in various forms, which can often be heard in the media and many of other traditional professions and beyond, although this accent is not limited to London and South East England, and can also be heard selectively throughout the whole UK amongst certain social groupings. Since the turn of the century the Cockney dialect is less common in the East End and has 'migrated' east to Havering and the county of Essex.[262][263] Economy Main article: Economy of London The City of London, one of the largest financial centres in the world[264] London produced in 2016 was £408 billion or $534 billion, over 22% of UK GDP,[265] while the economy of the London metropolitan area—the largest in Europe—generates about 30 per cent of the UK's GDP (or an estimated $669 billion in 2005).[266] London has five major business districts: the City, Westminster, Canary Wharf, Camden & Islington and Lambeth & Southwark. One way to get an idea of their relative importance is to look at relative amounts of office space: Greater London had 27 million m2 of office space in 2001, and the City contains the most space, with 8 million m2 of office space. London has some of the highest real estate prices in the world.[267][268] London is the world's most expensive office market for the last three years according to world property journal (2015) report.[269] As of 2015 the residential property in London is worth $2.2 trillion – same value as that of Brazil annual GDP.[270] The city has the highest property prices of any European city according to the Office for National Statistics and the European Office of Statistics.[271] On average the price per square metre in central London is €24,252 (April 2014). This is higher than the property prices in other G8 European capital cities; Berlin €3,306, Rome €6,188 and Paris €11,229.[272] The City of London The London Stock Exchange at Paternoster Square and Temple Bar London's finance industry is based in the City of London and Canary Wharf, the two major business districts in London. London is one of the pre-eminent financial centres of the world as the most important location for international finance.[273][274] London took over as a major financial centre shortly after 1795 when the Dutch Republic collapsed before the Napoleonic armies. For many bankers established in Amsterdam (e.g. Hope, Baring), this was only time to move to London. The London financial elite was strengthened by a strong Jewish community from all over Europe capable of mastering the most sophisticated financial tools of the time.[111] This unique concentration of talents accelerated the transition from the Commercial Revolution to the Industrial Revolution. By the end of the 19th century, Britain was the wealthiest of all nations, and London a leading financial centre. Still, as of 2016 London tops the world rankings on the Global Financial Centres Index (GFCI),[275] and it ranked second in A.T. Kearney's 2018 Global Cities Index.[276] London's largest industry is finance, and its financial exports make it a large contributor to the UK's balance of payments. Around 325,000 people were employed in financial services in London until mid-2007. London has over 480 overseas banks, more than any other city in the world. It is also the world's biggest currency trading centre, accounting for some 37 per cent of the $5.1 trillion average daily volume, according to the BIS.[277] Over 85 per cent (3.2 million) of the employed population of greater London works in the services industries. Because of its prominent global role, London's economy had been affected by the financial crisis of 2007–2008. However, by 2010 the City has recovered; put in place new regulatory powers, proceeded to regain lost ground and re-established London's economic dominance.[278] Along with professional services headquarters, the City of London is home to the Bank of England, London Stock Exchange, and Lloyd's of London insurance market. Over half of the UK's top 100 listed companies (the FTSE 100) and over 100 of Europe's 500 largest companies have their headquarters in central London. Over 70 per cent of the FTSE 100 are within London's metropolitan area, and 75 per cent of Fortune 500 companies have offices in London.[279] Media and technology Media companies are concentrated in London and the media distribution industry is London's second most competitive sector.[280] The BBC is a significant employer, while other broadcasters also have headquarters around the City. Many national newspapers are edited in London. London is a major retail centre and in 2010 had the highest non-food retail sales of any city in the world, with a total spend of around £64.2 billion.[281] The Port of London is the second-largest in the United Kingdom, handling 45 million tonnes of cargo each year.[282] A growing number of technology companies are based in London notably in East London Tech City, also known as Silicon Roundabout. In April 2014, the city was among the first to receive a geoTLD.[283] In February 2014 London was ranked as the European City of the Future [284] in the 2014/15 list by FDi Magazine.[285] The gas and electricity distribution networks that manage and operate the towers, cables and pressure systems that deliver energy to consumers across the city are managed by National Grid plc, SGN[286] and UK Power Networks.[287] Tourism Main article: Tourism in London The British Museum The National Gallery London is one of the leading tourist destinations in the world and in 2015 was ranked as the most visited city in the world with over 65 million visits.[288][289] It is also the top city in the world by visitor cross-border spending, estimated at US$20.23 billion in 2015.[290] Tourism is one of London's prime industries, employing the equivalent of 350,000 full-time workers in 2003,[291] and the city accounts for 54% of all inbound visitor spending in the UK.[292] As of 2016 London is the world top city destination as ranked by TripAdvisor users.[293] In 2015 the top most-visited attractions in the UK were all in London. The top 10 most visited attractions were: (with visits per venue) [294] The British Museum: 6,820,686 The National Gallery: 5,908,254 The Natural History Museum (South Kensington): 5,284,023 The Southbank Centre: 5,102,883 Tate Modern: 4,712,581 The Victoria and Albert Museum (South Kensington): 3,432,325 The Science Museum: 3,356,212 Somerset House: 3,235,104 The Tower of London: 2,785,249 The National Portrait Gallery: 2,145,486 The number of hotel rooms in London in 2015 stood at 138,769, and is expected to grow over the years.[295] Transport Main articles: Transport in London and Infrastructure in London Black London taxis Transport is one of the four main areas of policy administered by the Mayor of London,[296] however the mayor's financial control does not extend to the longer distance rail network that enters London. In 2007 he assumed responsibility for some local lines, which now form the London Overground network, adding to the existing responsibility for the London Underground, trams and buses. The public transport network is administered by Transport for London (TFL). The lines that formed the London Underground, as well as trams and buses, became part of an integrated transport system in 1933 when the London Passenger Transport Board or London Transport was created. Transport for London is now the statutory corporation responsible for most aspects of the transport system in Greater London, and is run by a board and a commissioner appointed by the Mayor of London.[297] Aviation Main article: Airports of London London Heathrow Airport is the busiest airport in Europe as well as the second busiest in the world for international passenger traffic. (Terminal 5C is pictured) London is a major international air transport hub with the busiest city airspace in the world. Eight airports use the word London in their name, but most traffic passes through six of these. Additionally, various other airports also serve London, catering primarily to general aviation flights. London Heathrow Airport, in Hillingdon, West London, was for many years the busiest airport in the world for international traffic, and is the major hub of the nation's flag carrier, British Airways.[298] In March 2008 its fifth terminal was opened.[299] In 2014, Dubai gained from Heathrow the leading position in terms of international passenger traffic.[300] London Gatwick Airport,[301] south of London in West Sussex, handles flights to more destinations than any other UK airport[302] and is the main base of easyJet,[303] the UK's largest airline by number of passengers.[304] London Stansted Airport,[305] north-east of London in Essex, has flights that serve the greatest number of European destinations of any UK airport[306] and is the main base of Ryanair,[307] the world's largest international airline by number of international passengers.[308] London Luton Airport, to the north of London in Bedfordshire, is used by several budget airlines for short-haul flights.[309] London City Airport, the most central airport and the one with the shortest runway, in Newham, East London, is focused on business travellers, with a mixture of full-service short-haul scheduled flights and considerable business jet traffic.[310] London Southend Airport, east of London in Essex, is a smaller, regional airport that caters for short-haul flights on a limited, though growing, number of airlines.[311] In 2017, international passengers made up over 95% of the total at Southend, the highest proportion of any London airport.[312] Rail Underground and DLR The London Underground is the world's oldest and third-longest rapid transit system The London Underground, commonly referred to as the Tube, is the oldest[313] and third longest[314] metro system in the world. The system serves 270 stations[315] and was formed from several private companies, including the world's first underground electric line, the City and South London Railway.[316] It dates from 1863.[317] Over four million journeys are made every day on the Underground network, over 1 billion each year.[318] An investment programme is attempting to reduce congestion and improve reliability, including £6.5 billion (€7.7 billion) spent before the 2012 Summer Olympics.[319] The Docklands Light Railway (DLR), which opened in 1987, is a second, more local metro system using smaller and lighter tram-type vehicles that serve the Docklands, Greenwich and Lewisham. Suburban A Thameslink train at Blackfriars There are more than 360 railway stations in the London Travelcard Zones on an extensive above-ground suburban railway network. South London, particularly, has a high concentration of railways as it has fewer Underground lines. Most rail lines terminate around the centre of London, running into eighteen terminal stations, with the exception of the Thameslink trains connecting Bedford in the north and Brighton in the south via Luton and Gatwick airports.[320] London has Britain's busiest station by number of passengers – Waterloo, with over 184 million people using the interchange station complex (which includes Waterloo East station) each year.[321][322] Clapham Junction is the busiest station in Europe by the number of trains passing. With the need for more rail capacity in London, Crossrail is due to open in 2018. It will be a new railway line running east to west through London and into the Home Counties with a branch to Heathrow Airport.[323] It is Europe's biggest construction project, with a £15 billion projected cost.[324][325] Inter-city and international St Pancras International is the main terminal for high speed Eurostar and HS1 services, as well as commuter suburban Thameslink and inter-city East Midlands Trains services London is the centre of the National Rail network, with 70 per cent of rail journeys starting or ending in London.[326] Like suburban rail services, regional and inter-city trains depart from several termini around the city centre, linking London with the rest of Britain including Birmingham, Brighton, Reading, Bristol, Cardiff, Derby, Exeter, Sheffield, Southampton, Leeds, Liverpool, Manchester, Cambridge, Newcastle upon Tyne, Edinburgh and Glasgow. Some international railway services to Continental Europe were operated during the 20th century as boat trains, such as the Admiraal de Ruijter to Amsterdam and the Night Ferry to Paris and Brussels. The opening of the Channel Tunnel in 1994 connected London directly to the continental rail network, allowing Eurostar services to begin. Since 2007, high-speed trains link St. Pancras International with Lille, Paris, Brussels, Amsterdam and other European tourist destinations via the High Speed 1 rail link and the Channel Tunnel.[327] The first high-speed domestic trains started in June 2009 linking Kent to London.[328] There are plans for a second high speed line linking London to the Midlands, North West England, and Yorkshire. Freight Although rail freight levels are far down compared to their height, significant quantities of cargo are also carried into and out of London by rail; chiefly building materials and landfill waste.[329] As a major hub of the British railway network, London's tracks also carry large amounts of freight for the other regions, such as container freight from the Channel Tunnel and English Channel ports, and nuclear waste for reprocessing at Sellafield.[329] Buses and trams The red double decker bus is an iconic symbol of London London's bus network is one of the largest in the world, running 24 hours a day, with about 8,500 buses, more than 700 bus routes and around 19,500 bus stops.[330] In 2013, the network had more than 2 billion commuter trips per annum, more than the Underground.[330] Around £850 million is taken in revenue each year. London has the largest wheelchair accessible network in the world[331] and, from the 3rd quarter of 2007, became more accessible to hearing and visually impaired passengers as audio-visual announcements were introduced. The distinctive red double-decker buses are an internationally recognised trademark of London transport along with black cabs and the Tube.[332][333] London has a modern tram network, known as Tramlink, centred on Croydon in South London. The network has 39 stops and four routes, and carried 28 million people in 2013.[334] Since June 2008 Transport for London has completely owned Tramlink, and it plans to spend £54m by 2015 on maintenance, renewals, upgrades and capacity enhancements.[335] Cable car London's first and only cable car, known as the Emirates Air Line, opened in June 2012. Crossing the River Thames, linking Greenwich Peninsula and the Royal Docks in the east of the city, the cable car is integrated with London's Oyster Card ticketing system, although special fares are charged. Costing £60 million to build, it carries over 3,500 passengers every day, although this is very much lower than its capacity. Similar to the Santander Cycles bike hire scheme, the cable car is sponsored in a 10-year deal by the airline Emirates. Cycling Main article: Cycling in London Santander Cycle Hire near Victoria in Central London Segregated cycle lanes are being implemented across London. Cycle Superhighway 2 in Stratford In the whole Greater London Area, around 650,000 people use a bike everyday.[336] But out of a total population of around 8.8 million,[337] this means that just around 7% of Greater London's population use a bike on an average day. This is a small proportion, when compared to many other cities in the world [338] A reason may well be the poor investments for cycling in London of about £110 million per year,[339] equating to around £12 per person, which can be compared to £22 in the Netherlands.[340] Cycling is nevertheless becoming increasingly popular way to get around London. The launch of a cycle hire scheme in July 2010 has been successful and generally well received. The London Cycling Campaign lobbies for better provision.[341] There are many cycle routes in London, including several Cycle Superhighways. Port and river boats From being the largest port in the world, the Port of London is now only the second-largest in the United Kingdom, handling 45 million tonnes of cargo each year.[282] Most of this actually passes through the Port of Tilbury, outside the boundary of Greater London. London has frequent river boat services on the Thames known as Thames Clippers. These run up to every 20 minutes between Embankment Pier and North Greenwich Pier. The Woolwich Ferry, with 2.5 million passengers every year,[342] is a frequent service linking the North and South Circular Roads. Other operators run both commuter and tourist boat services in London. Roads Although the majority of journeys involving central London are made by public transport, car travel is common in the suburbs. The inner ring road (around the city centre), the North and South Circular roads (in the suburbs), and the outer orbital motorway (the M25, outside the built-up area) encircle the city and are intersected by a number of busy radial routes—but very few motorways penetrate into inner London. A plan for a comprehensive network of motorways throughout the city (the Ringways Plan) was prepared in the 1960s but was mostly cancelled in the early 1970s. The M25 is the second-longest ring-road motorway in Europe at 117 mi (188 km) long.[343] The A1 and M1 connect London to Leeds, and Newcastle and Edinburgh. London is notorious for its traffic congestion, with the average speed of a car in the rush hour being 10.6 mph (17.1 km/h).[344] In 2003, a congestion charge was introduced to reduce traffic volumes in the city centre. With a few exceptions, motorists are required to pay £10 per day to drive within a defined zone encompassing much of central London.[345][346] Motorists who are residents of the defined zone can buy a greatly reduced season pass.[347] London government initially expected the Congestion Charge Zone to increase daily peak period Underground and bus users by 20,000 people, reduce road traffic by 10 to 15 per cent, increase traffic speeds by 10 to 15 per cent, and reduce queues by 20 to 30 per cent.[348] Over the course of several years, the average number of cars entering the centre of London on a weekday was reduced from 195,000 to 125,000 cars – a 35-per-cent reduction of vehicles driven per day.[349] Education Main article: Education in London Tertiary education Imperial College London, a world leading research university located in South Kensington London is a major global centre of higher education teaching and research and has the largest concentration of higher education institutes in Europe.[52] According to the QS World University Rankings 2015/16, London has the greatest concentration of top class universities in the world[350][351] and its international student population of around 110,000 is larger than any other city in the world.[352] A 2014 PricewaterhouseCoopers report termed London as the global capital of higher education.[353] A number of world-leading education institutions are based in London. In the 2014/15 QS World University Rankings, Imperial College London is ranked joint 2nd in the world, University College London (UCL) is ranked 5th, and King's College London (KCL) is ranked 16th.[354] The London School of Economics has been described as the world's leading social science institution for both teaching and research.[355] The London Business School is considered one of the world's leading business schools and in 2015 its MBA programme was ranked second best in the world by the Financial Times.[356] With 120,000 students in London,[357] the federal University of London is the largest contact teaching university in the UK.[358] It includes five multi-faculty universities – City, King's College London, Queen Mary, Royal Holloway and UCL – and a number of smaller and more specialised institutions including Birkbeck, the Courtauld Institute of Art, Goldsmiths, Guildhall School of Music and Drama, the London Business School, the London School of Economics, the London School of Hygiene & Tropical Medicine, the Royal Academy of Music, the Central School of Speech and Drama, the Royal Veterinary College and the School of Oriental and African Studies.[359] Members of the University of London have their own admissions procedures, and some award their own degrees. A number of universities in London are outside the University of London system, including Brunel University, Imperial College London, Kingston University, London Metropolitan University,[360] University of East London, University of West London, University of Westminster, London South Bank University, Middlesex University, and University of the Arts London (the largest university of art, design, fashion, communication and the performing arts in Europe).[361] In addition there are three international universities in London – Regent's University London, Richmond, The American International University in London and Schiller International University. The front façade of the Royal College of Music London is home to five major medical schools – Barts and The London School of Medicine and Dentistry (part of Queen Mary), King's College London School of Medicine (the largest medical school in Europe), Imperial College School of Medicine, UCL Medical School and St George's, University of London – and has a large number of affiliated teaching hospitals. It is also a major centre for biomedical research, and three of the UK's eight academic health science centres are based in the city – Imperial College Healthcare, King's Health Partners and UCL Partners (the largest such centre in Europe).[362] There are a number of business schools in London, including the London School of Business and Finance, Cass Business School (part of City University London), Hult International Business School, ESCP Europe, European Business School London, Imperial College Business School, the London Business School and the UCL School of Management. London is also home to many specialist arts education institutions, including the Academy of Live and Recorded Arts, Central School of Ballet, LAMDA, London College of Contemporary Arts (LCCA), London Contemporary Dance School, National Centre for Circus Arts, RADA, Rambert School of Ballet and Contemporary Dance, the Royal College of Art, the Royal College of Music and Trinity Laban. King's College London, established by Royal Charter having been founded by King George IV and the Duke of Wellington in 1829, is one of the founding colleges of the University of London Primary and secondary education The majority of primary and secondary schools and further-education colleges in London are controlled by the London boroughs or otherwise state-funded; leading examples include Ashbourne College, Bethnal Green Academy, Brampton Manor Academy, City and Islington College, City of Westminster College, David Game College, Ealing, Hammersmith and West London College, Leyton Sixth Form College, London Academy of Excellence, Tower Hamlets College, and Newham Collegiate Sixth Form Centre. There are also a number of private schools and colleges in London, some old and famous, such as City of London School, Harrow, St Paul's School, Haberdashers' Aske's Boys' School, University College School, The John Lyon School, Highgate School and Westminster School. Culture Main article: Culture of London Leisure and entertainment See also: List of annual events in London and West End theatre Piccadilly Circus Leisure is a major part of the London economy, with a 2003 report attributing a quarter of the entire UK leisure economy to London[363] at 25.6 events per 1000 people.[364] Globally, the city is amongst the big four fashion capitals of the world, and according to official statistics, London is the world's third busiest film production centre, presents more live comedy than any other city,[365] and has the biggest theatre audience of any city in the world.[366] Harrods in Knightsbridge Within the City of Westminster in London, the entertainment district of the West End has its focus around Leicester Square, where London and world film premieres are held, and Piccadilly Circus, with its giant electronic advertisements.[367] London's theatre district is here, as are many cinemas, bars, clubs, and restaurants, including the city's Chinatown district (in Soho), and just to the east is Covent Garden, an area housing speciality shops. The city is the home of Andrew Lloyd Webber, whose musicals have dominated the West End theatre since the late 20th century.[368] The United Kingdom's Royal Ballet, English National Ballet, Royal Opera, and English National Opera are based in London and perform at the Royal Opera House, the London Coliseum, Sadler's Wells Theatre, and the Royal Albert Hall, as well as touring the country.[369] Scene of the annual Notting Hill Carnival, 2014 Islington's 1 mile (1.6 km) long Upper Street, extending northwards from Angel, has more bars and restaurants than any other street in the United Kingdom.[370] Europe's busiest shopping area is Oxford Street, a shopping street nearly 1 mile (1.6 km) long, making it the longest shopping street in the UK. Oxford Street is home to vast numbers of retailers and department stores, including the world-famous Selfridges flagship store.[371] Knightsbridge, home to the equally renowned Harrods department store, lies to the south-west. London is home to designers Vivienne Westwood, Galliano, Stella McCartney, Manolo Blahnik, and Jimmy Choo, among others; its renowned art and fashion schools make it an international centre of fashion alongside Paris, Milan, and New York City. London offers a great variety of cuisine as a result of its ethnically diverse population. Gastronomic centres include the Bangladeshi restaurants of Brick Lane and the Chinese restaurants of Chinatown.[372] Shakespeare's Globe is a modern reconstruction of the Globe Theatre on the south bank of the River Thames There is a variety of annual events, beginning with the relatively new New Year's Day Parade, a fireworks display at the London Eye; the world's second largest street party, the Notting Hill Carnival, is held on the late August Bank Holiday each year. Traditional parades include November's Lord Mayor's Show, a centuries-old event celebrating the annual appointment of a new Lord Mayor of the City of London with a procession along the streets of the City, and June's Trooping the Colour, a formal military pageant performed by regiments of the Commonwealth and British armies to celebrate the Queen's Official Birthday.[373] Literature, film and television Main articles: London in fiction, London in film, List of television shows set in London, and London Television Archive Sherlock Holmes Museum in Baker Street, bearing the number 221B London has been the setting for many works of literature. The pilgrims in Geoffrey Chaucer's late 14th-century Canterbury Tales set out for Canterbury from London – specifically, from the Tabard inn, Southwark. William Shakespeare spent a large part of his life living and working in London; his contemporary Ben Jonson was also based there, and some of his work, most notably his play The Alchemist, was set in the city.[374] A Journal of the Plague Year (1722) by Daniel Defoe is a fictionalisation of the events of the 1665 Great Plague.[374] The literary centres of London have traditionally been hilly Hampstead and (since the early 20th century) Bloomsbury. Writers closely associated with the city are the diarist Samuel Pepys, noted for his eyewitness account of the Great Fire, Charles Dickens, whose representation of a foggy, snowy, grimy London of street sweepers and pickpockets has been a major influence on people's vision of early Victorian London, and Virginia Woolf, regarded as one of the foremost modernist literary figures of the 20th century.[374] Later important depictions of London from the 19th and early 20th centuries are Dickens' novels, and Arthur Conan Doyle's Sherlock Holmes stories.[374] Also of significance is Letitia Elizabeth Landon's Calendar of the London Seasons (1834). Modern writers pervasively influenced by the city include Peter Ackroyd, author of a "biography" of London, and Iain Sinclair, who writes in the genre of psychogeography. 	Wikisource has original text related to this article: 'Calendar of the London Seasons', by L. E. L. Keats House, where Keats wrote his Ode to a Nightingale. The village of Hampstead has historically been a literary centre in London. London has played a significant role in the film industry. Major studios within or bordering London include Twickenham, Ealing, Shepperton, Pinewood, Elstree and Borehamwood,[375] and a special effects and post-production community centred in Soho. Working Title Films has its headquarters in London.[376] London has been the setting for films including Oliver Twist (1948), Scrooge (1951), Peter Pan (1953), The 101 Dalmatians (1961), My Fair Lady (1964), Mary Poppins (1964), Blowup (1966), The Long Good Friday (1980), Notting Hill (1999), Love Actually (2003), V For Vendetta (2005), Sweeney Todd: The Demon Barber Of Fleet Street (2008) and The King's Speech (2010). Notable actors and filmmakers from London include; Charlie Chaplin, Alfred Hitchcock, Michael Caine, Helen Mirren, Gary Oldman, Christopher Nolan, Jude Law, Benedict Cumberbatch, Tom Hardy, Keira Knightley and Daniel Day-Lewis. As of 2008, the British Academy Film Awards have taken place at the Royal Opera House. London is a major centre for television production, with studios including BBC Television Centre, The Fountain Studios and The London Studios. Many television programmes have been set in London, including the popular television soap opera EastEnders, broadcast by the BBC since 1985. Museums and art galleries Aerial view of Albertopolis. Albert Memorial, Royal Albert Hall and Royal College of Art are visible near the top; Victoria and Albert Museum and Natural History Museum at the lower end; Imperial College, Royal College of Music, and Science Museum lying in between. London is home to many museums, galleries, and other institutions, many of which are free of admission charges and are major tourist attractions as well as playing a research role. The first of these to be established was the British Museum in Bloomsbury, in 1753. Originally containing antiquities, natural history specimens, and the national library, the museum now has 7 million artefacts from around the globe. In 1824, the National Gallery was founded to house the British national collection of Western paintings; this now occupies a prominent position in Trafalgar Square. In the latter half of the 19th century the locale of South Kensington was developed as "Albertopolis", a cultural and scientific quarter. Three major national museums are there: the Victoria and Albert Museum (for the applied arts), the Natural History Museum, and the Science Museum. The National Portrait Gallery was founded in 1856 to house depictions of figures from British history; its holdings now comprise the world's most extensive collection of portraits.[377] The national gallery of British art is at Tate Britain, originally established as an annexe of the National Gallery in 1897. The Tate Gallery, as it was formerly known, also became a major centre for modern art; in 2000, this collection moved to Tate Modern, a new gallery housed in the former Bankside Power Station. Music The Royal Albert Hall hosts concerts and musical events London is one of the major classical and popular music capitals of the world and hosts major music corporations, such as Universal Music Group International and Warner Music Group, as well as countless bands, musicians and industry professionals. The city is also home to many orchestras and concert halls, such as the Barbican Arts Centre (principal base of the London Symphony Orchestra and the London Symphony Chorus), Cadogan Hall (Royal Philharmonic Orchestra) and the Royal Albert Hall (The Proms).[369] London's two main opera houses are the Royal Opera House and the London Coliseum.[369] The UK's largest pipe organ is at the Royal Albert Hall. Other significant instruments are at the cathedrals and major churches. Several conservatoires are within the city: Royal Academy of Music, Royal College of Music, Guildhall School of Music and Drama and Trinity Laban. Abbey Road Studios, 3 Abbey Road, St John's Wood, City of Westminster London has numerous venues for rock and pop concerts, including the world's busiest arena the O2 arena[378] and Wembley Arena, as well as many mid-sized venues, such as Brixton Academy, the Hammersmith Apollo and the Shepherd's Bush Empire.[369] Several music festivals, including the Wireless Festival, South West Four, Lovebox, and Hyde Park's British Summer Time are all held in London.[379] The city is home to the original Hard Rock Cafe and the Abbey Road Studios, where The Beatles recorded many of their hits. In the 1960s, 1970s and 1980s, musicians and groups like Elton John, Pink Floyd, Cliff Richard, David Bowie, Queen, The Kinks, The Rolling Stones, The Who, Eric Clapton, Led Zeppelin, The Small Faces, Iron Maiden, Fleetwood Mac, Elvis Costello, Cat Stevens, The Police, The Cure, Madness, The Jam, Ultravox, Spandau Ballet, Culture Club, Dusty Springfield, Phil Collins, Rod Stewart, Adam Ant, Status Quo and Sade, derived their sound from the streets and rhythms of London.[380] London was instrumental in the development of punk music,[381] with figures such as the Sex Pistols, The Clash,[380] and Vivienne Westwood all based in the city. More recent artists to emerge from the London music scene include George Michael's Wham!, Kate Bush, Seal, the Pet Shop Boys, Bananarama, Siouxsie and the Banshees, Bush, the Spice Girls, Jamiroquai, Blur, McFly, The Prodigy, Gorillaz, Bloc Party, Mumford & Sons, Coldplay, Amy Winehouse, Adele, Sam Smith, Ed Sheeran, Paloma Faith, Ellie Goulding, One Direction and Florence and the Machine.[382][383][384] London is also a centre for urban music. In particular the genres UK garage, drum and bass, dubstep and grime evolved in the city from the foreign genres of hip hop and reggae, alongside local drum and bass. Music station BBC Radio 1Xtra was set up to support the rise of local urban contemporary music both in London and in the rest of the United Kingdom. Notable people Main article: List of people from London Recreation Parks and open spaces Main articles: Parks and open spaces in London and Royal Parks of London See also: List of Sites of Special Scientific Interest in Greater London and List of Local Nature Reserves in Greater London Hyde Park Aerial view of Hyde Park Lake with London Eye in the background St. James's Park lake with the London Eye in the distance A 2013 report by the City of London Corporation said that London is the "greenest city" in Europe with 35,000 acres of public parks, woodlands and gardens.[385] The largest parks in the central area of London are three of the eight Royal Parks, namely Hyde Park and its neighbour Kensington Gardens in the west, and Regent's Park to the north.[386] Hyde Park in particular is popular for sports and sometimes hosts open-air concerts. Regent's Park contains London Zoo, the world's oldest scientific zoo, and is near Madame Tussauds Wax Museum.[387][388] Primrose Hill, immediately to the north of Regent's Park, at 256 feet (78 m)[389] is a popular spot from which to view the city skyline. Close to Hyde Park are smaller Royal Parks, Green Park and St. James's Park.[390] A number of large parks lie outside the city centre, including Hampstead Heath and the remaining Royal Parks of Greenwich Park to the southeast[391] and Bushy Park and Richmond Park (the largest) to the southwest,[392][393] Hampton Court Park is also a royal park, but, because it contains a palace, it is administered by the Historic Royal Palaces, unlike the eight Royal Parks.[394] Close to Richmond Park is Kew Gardens which has the world's largest collection of living plants. In 2003, the gardens were put on the UNESCO list of World Heritage Sites.[395] There are also parks administered by London's borough Councils, including Victoria Park in the East End and Battersea Park in the centre. Some more informal, semi-natural open spaces also exist, including the 320-hectare (790-acre) Hampstead Heath of North London,[396] and Epping Forest, which covers 2,476 hectares (6,118 acres)[397] in the east. Both are controlled by the City of London Corporation.[398][399] Hampstead Heath incorporates Kenwood House, a former stately home and a popular location in the summer months when classical musical concerts are held by the lake, attracting thousands of people every weekend to enjoy the music, scenery and fireworks.[400] Epping Forest is a popular venue for various outdoor activities, including mountain biking, walking, horse riding, golf, angling, and orienteering.[401] Walking Walking is a popular recreational activity in London. Areas that provide for walks include Wimbledon Common, Epping Forest, Hampton Court Park, Hampstead Heath, the eight Royal Parks, canals and disused railway tracks.[402] Access to canals and rivers has improved recently, including the creation of the Thames Path, some 28 miles (45 km) of which is within Greater London, and The Wandle Trail; this runs 12 miles (19 km) through South London along the River Wandle, a tributary of the River Thames.[403] Other long distance paths, linking green spaces, have also been created, including the Capital Ring, the Green Chain Walk, London Outer Orbital Path ("Loop"), Jubilee Walkway, Lea Valley Walk, and the Diana, Princess of Wales Memorial Walk.[404] Sport Main article: Sport in London Wembley Stadium, home of the England football team, has a 90,000 capacity. It is the UK's biggest stadium. London has hosted the Summer Olympics three times: in 1908, 1948, and 2012.[405][406] making it the first city to host the modern Games three times.[53] The city was also the host of the British Empire Games in 1934.[407] In 2017, London hosted the World Championships in Athletics for the first time.[408] London's most popular sport is football and it has six clubs in the English Premier League: Arsenal, Chelsea, Crystal Palace, Fulham, Tottenham Hotspur, and West Ham United.[409] Other professional teams in London are Queens Park Rangers, Brentford, Millwall, Charlton Athletic, AFC Wimbledon and Barnet. Twickenham, home of the England rugby union team, has an 82,000 capacity, the world's largest rugby union stadium From 1924, the original Wembley Stadium was the home of the English national football team. It hosted the 1966 FIFA World Cup Final, with England defeating West Germany, and served as the venue for the FA Cup Final as well as rugby league's Challenge Cup final.[410] The new Wembley Stadium serves exactly the same purposes and has a capacity of 90,000.[411] Two Aviva Premiership rugby union teams are based in London, Saracens and Harlequins.[412] London Scottish, London Welsh and London Irish play in the RFU Championship club and other rugby union clubs in the city include Richmond F.C., Rosslyn Park F.C., Westcombe Park R.F.C. and Blackheath F.C.. Twickenham Stadium in south-west London hosts home matches for the England national rugby union team and has a capacity of 82,000 now that the new south stand has been completed.[413] Centre Court at Wimbledon. First played in 1877, the Championships is the oldest tennis tournament in the world.[414] While rugby league is more popular in the north of England, there are two professional rugby league clubs in London – the second tier Championship One team, the London Broncos, who play at the Trailfinders Sports Ground in West Ealing, and the third tier League 1 team, the London Skolars from Wood Green, Haringey; in addition, Hemel Stags from Hemel Hempstead north of London also play in League 1. One of London's best-known annual sports competitions is the Wimbledon Tennis Championships, held at the All England Club in the south-western suburb of Wimbledon.[415] Played in late June to early July, it is the oldest tennis tournament in the world, and widely considered the most prestigious.[416][417][418] London has two Test cricket grounds, Lord's (home of Middlesex C.C.C.) in St John's Wood[419] and the Oval (home of Surrey C.C.C.) in Kennington.[420] Lord's has hosted four finals of the Cricket World Cup. Other key events are the annual mass-participation London Marathon, in which some 35,000 runners attempt a 26.2 miles (42.2 km) course around the city,[421] and the University Boat Race on the River Thames from Putney to Mortlake.[422].
"""

# Parse the document with spaCy
doc = nlp(text)

# Extract noun chunks that appear
noun_chunks = textacy.extract.noun_chunks(doc, min_freq=3)

# Convert noun chunks to lowercase strings
noun_chunks = map(str, noun_chunks)
noun_chunks = map(str.lower, noun_chunks)

# Print out any nouns that are at least 2 words long
for noun_chunk in set(noun_chunks):
    if len(noun_chunk.split(" ")) > 1:
        print(noun_chunk)

epping forest
south kensington
new york city
london underground
london's population
hampstead heath
tate modern
central london
2011 census
south london
royal exchange
united kingdom
victoria and albert museum
population density
natural history museum
european union
river thames
west end
king's college london
large number
second world war
london school
st paul's cathedral
city centre
greater london's population
royal albert hall
office space
west london
trafalgar square
major centre
other city
greater london
square kilometre
national gallery
london business school
inner london
london eye
british museum
south bank
royal opera house
east end
great fire
regent's park
royal college
tower bridge
national statistics
canary wharf
